In [1]:
def intersection_of_lists(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    intersection = set1.intersection(set2)
    intersection_list = sorted(list(intersection))
    return intersection_list

list1 = [1, 2, 3, 4, 5]
list2 = [4, 5, 6, 7]
print(intersection_of_lists(list1, list2))  # Output: [4, 5]

[4, 5]
